In [ ]:
# !pip install transformers==4.8.2

# Grammar Error Corection

In [77]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
# model_name = 'deep-learning-analytics/GrammarCorrector'
model_name = 't5_gec_model_02' # model path
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name).to(torch_device)

def correct_grammar(input_text,num_return_sequences):
  batch = tokenizer([input_text],truncation=True,padding='max_length',max_length=64, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=64,num_beams=4, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

In [84]:
input_text = "They can find new customers and introduce their services to each other. For instance, they commercials between the shows certify this issue.Furthermore TV is one of the tools that entertain people, although the target finds gift, so part of this shows for entertaining people."
num_return_sequences = 4 # Số lượng câu generate
corrected_text = correct_grammar(input_text, num_return_sequences)
corrected_text

['They can find new customers and introduce their services to each other. For instance, the commercials between the shows certify this issue.Furthermore, TV is one of the tools that entertain people, although the target finds gift, so part of this shows for entertaining people.',
 'They can find new customers and introduce their services to each other. For instance, they commercials between the shows certify this issue.Furthermore, TV is one of the tools that entertain people, although the target finds gift, so part of this shows for entertaining people.',
 'They can find new customers and introduce their services to each other. For instance, the commercials between the shows certify this issue. Further, TV is one of the tools that entertain people, although the target finds gift, so part of this shows for entertaining people.',
 'They can find new customers and introduce their services to each other. For instance, the commercials between the shows certify this issue.Furthermore TV is 

# Question Generation and Fact - Check

In [1]:
# import fitz  # PyMuPDF
from transformers import T5ForConditionalGeneration, T5Tokenizer

model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

c:\Users\Sysme Hue\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Sysme Hue\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Sysme Hue\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In ord

In [21]:
import re
import textract

# Đọc nội dung của tài liệu PDF
text = textract.process('pdf/Report.pdf', encoding='utf-8')

# Sử dụng biểu thức chính quy để cắt thành các đoạn văn
all_paragraphs = re.split(r'\s{2,}', text.decode('utf-8'))
num_paragraph= len(text)
print(num_paragraph)

13876


In [22]:
list_para = []
list_para = [para for para in all_paragraphs if len(para.split()) >= 20] # list of paragraphs which have more than 20 words
len(list_para)

21

In [23]:
# Summarize paragraph using t5 model

summaries = []
for i,paragraph in enumerate(list_para):
    input_text = "summarize: " + paragraph
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(input_ids, max_length=150, min_length=50, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    summaries.append(summary)
    
    # if i==3: break

In [24]:
list_para[:4]

['Abstract. This paper aims to develop a system that will help in recommendation of courses for an upcoming semester based on the performance of previous semesters.',
 "It has always been a tough choice for the students to choose the courses in different semesters in which there is possibility to score good grades apart from the interest in the course. IIIT-Delhi offers variety of courses with mandatory courses in first 4 semesters (with exception of 2 to 3 electives) and all elective courses from fifth semester onwards. Hence, choosing the courses based on the verbal recommendation from the seniors, instructors and fellowmates becomes a hectic task. For easing this process of course recommendation for an upcoming semester, we have developed a system which deploys simple yet powerful recommendation techniques such as auto-encoders, hybrid matrix factorization and similarity based approaches. It is a GUI based system which takes an input of student's ID (which is stored in the backend d

In [25]:
# merge into result 

result = [{"paragraph": list_para[i], "summary": summaries[i]} for i in range(len(summaries))]

In [26]:
result[:5]

[{'paragraph': 'Abstract. This paper aims to develop a system that will help in recommendation of courses for an upcoming semester based on the performance of previous semesters.',
  'summary': 'this paper aims to develop a system that will help in recommendation of courses for an upcoming semester based on the performance of previous semesters. this paper aims to develop a system that will help in recommendation of courses based on the performance of previous semesters.'},
 {'paragraph': "It has always been a tough choice for the students to choose the courses in different semesters in which there is possibility to score good grades apart from the interest in the course. IIIT-Delhi offers variety of courses with mandatory courses in first 4 semesters (with exception of 2 to 3 electives) and all elective courses from fifth semester onwards. Hence, choosing the courses based on the verbal recommendation from the seniors, instructors and fellowmates becomes a hectic task. For easing this

In [87]:
# Clone this responsitory for questions generator

# !git clone https://github.com/amontgomerie/question_generator
# !pip install -r question_generator/requirements.txt -qq
!python run_qg.py --text_file question_generator/articles/twitter_hack.txt

python: can't open file 'c:\\Users\\Sysme Hue\\Desktop\\LMS\\Libs\\run_qg.py': [Errno 2] No such file or directory


In [90]:
%cd question_generator

c:\Users\Sysme Hue\Desktop\LMS\Libs\question_generator


In [92]:
# import file from another folder

# import sys
# # caution: path[0] is reserved for script path (or '' in REPL)
# sys.path.insert(1, 'question_generator')

from questiongenerator import QuestionGenerator


In [ ]:
qg = QuestionGenerator()

Found existing installation: transformers 4.30.2
Uninstalling transformers-4.30.2:
  Would remove:
    c:\users\sysme hue\appdata\local\programs\python\python310\lib\site-packages\transformers-4.30.2.dist-info\*
    c:\users\sysme hue\appdata\local\programs\python\python310\lib\site-packages\transformers\*
    c:\users\sysme hue\appdata\local\programs\python\python310\scripts\transformers-cli.exe
Proceed (Y/n)? 


In [36]:
text = result[2]["summary"]
q  = qg.generate(text, num_questions=3)

Generating questions...



c:\Users\Sysme Hue\AppData\Local\Programs\Python\Python310\lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.1.0) was trained with spaCy v3.1.0 and may not be 100% compatible with the current version (3.7.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
c:\Users\Sysme Hue\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\generation\utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Evaluating QA pairs...



In [37]:
q

[{'question': 'how many students have taken the course?',
  'answer': 'the dataset consists 739 students and 306 subjects with mapping of each student to the grades for each course the student has taken throughout the duration of their degree.'},
 {'question': 'how many students have taken the iitt?',
  'answer': 'the dataset has been acquired from the official IIIT-Delhi academics department for the students of 7 Computer Science passout batches.'},
 {'question': 'how many subjects do students have taken?',
  'answer': [{'answer': 'dataset', 'correct': False},
   {'answer': '7 Computer Science', 'correct': False},
   {'answer': '306', 'correct': True},
   {'answer': '739', 'correct': False}]}]

In [38]:
print(text)
for i in range(len(q)): print(q[i]["question"]) #  7 Computer Science passout batches.

the dataset has been acquired from the official IIIT-Delhi academics department for the students of 7 Computer Science passout batches. the dataset consists 739 students and 306 subjects with mapping of each student to the grades for each course the student has taken throughout the duration of their degree.
how many students have taken the course?
how many students have taken the iitt?
how many subjects do students have taken?


### Fact -check

In [17]:
import torch
import torch.nn as nn

from transformers import BertTokenizer, BertModel

c:\Users\Sysme Hue\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Sysme Hue\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\tensorboard\__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(
c:\Users\Sysme Hue\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\tensorboard\__init__.py:6: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  ) < LooseVersion("1.15"):


In [18]:
# BERT MODEL

class BERTClassificationModel(nn.Module):
    def __init__(self, bert_model_name, num_labels):
        super(BERTClassificationModel, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(self.bert.config.hidden_size * 2, num_labels)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = torch.cat((outputs.last_hidden_state[:, 0, :], outputs.last_hidden_state[:, -1, :]), dim=1)
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

In [29]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

In [24]:
import torch
model_path = "model/bert_classification_model.pth"
# Tạo mô hình mới
loaded_model = BERTClassificationModel('bert-base-uncased', num_labels=3)


In [26]:
if torch.cuda.is_available():
    loaded_model.load_state_dict(torch.load(model_path))
else:
    loaded_model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

In [27]:
# Tải lại tokenizer từ đường dẫn đã lưu
loaded_tokenizer = BertTokenizer.from_pretrained("model/tokenizer")

In [30]:
max_seq_length = 128
def predict_premise_hypothesis(premise_text, hypothesis_text, model, tokenizer):
    # Chuẩn bị dữ liệu đầu vào cho mô hình
    inputs = tokenizer(premise_text, hypothesis_text, padding=True, truncation=True, max_length=max_seq_length, return_tensors="pt")
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)

    # Dự đoán
    model.eval()
    with torch.no_grad():
        logits = model(input_ids, attention_mask)
        probabilities = torch.softmax(logits, dim=1)
        predicted_label = torch.argmax(probabilities, dim=1).item()

    return predicted_label

pre = ["entailment", "neutral", "contradiction"]
# Sử dụng hàm predict_premise_hypothesis để dự đoán

premise_text = "Yesterday, I bought a new computer"
hypothesis_text = "I did not buy a new computer"
predicted_label = predict_premise_hypothesis(premise_text, hypothesis_text, loaded_model, loaded_tokenizer)

print("premise_text :", premise_text, "\n", "hypothesis_text :", hypothesis_text)
print("Predicted Label:", pre[predicted_label])


premise_text : Yesterday, I bought a new computer 
 hypothesis_text : I did not buy a new computer
Predicted Label: contradiction


In [54]:
pre = ["entailment", "neutral", "contradiction"]
# Sử dụng hàm predict_premise_hypothesis để dự đoán
premise_text = "The likelihood is 100%"
hypothesis_text = "Maybe the probability is 100%"
predicted_label = predict_premise_hypothesis(premise_text, hypothesis_text, loaded_model, loaded_tokenizer)

print("premise_text :", premise_text, "\n", "hypothesis_text :", hypothesis_text)
print("Predicted Label:", pre[predicted_label])

premise_text : The likelihood is 100% 
 hypothesis_text : Maybe the probability is 100%
Predicted Label: neutral


In [ ]:
premise_text = "The dataset had acquired for the student of 7 computer science"
hypothesis_text = result[2]["summary"]

predicted_label = predict_premise_hypothesis(premise_text, hypothesis_text, loaded_model, loaded_tokenizer)
print("premise_text :", premise_text, "\n", "hypothesis_text :", hypothesis_text)
print("Predicted Label:", pre[predicted_label])

## Find similar Course

In [31]:
import pandas as pd
import numpy as np
import re
from mlxtend.frequent_patterns import apriori, association_rules
import en_core_web_sm
spc_en = en_core_web_sm.load()

c:\Users\Sysme Hue\AppData\Local\Programs\Python\Python310\lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.1.0) was trained with spaCy v3.1.0 and may not be 100% compatible with the current version (3.7.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [32]:
course = pd.read_csv("../data/df_c.csv")
# id = pd.read_csv("../data/id.csv")

In [33]:
course_ids = [str(i).zfill(4) for i in range(1, len(course) + 1)]

# Thêm cột "Course_id" vào DataFrame
course.insert(0, 'Course_id', course_ids)

In [34]:
course.head()

,Course_id,Course Name,University,Difficulty Level,Course Rating,Course URL,Course Description,Skills,Specialized,Sequence,Vector,IDF,BERT_Encoded
0,0001,write feature length screenplay film television,Michigan State University,Beginner,4.8,https://www.coursera.org/learn/write-a-feature...,write full length feature film script course w...,drama comedy peer screenwrite film document re...,Arts and Humanities,write feature length screenplay film televisio...,[3.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 2.0 0.0 0.0 0.0],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[ 3.09796989e-01 8.47294629e-02 4.00437862e-...
1,0002,business strategy business model canvas analys...,Coursera Project Network,Beginner,4.8,https://www.coursera.org/learn/canvas-analysis...,end guide project fluent identifying create bu...,finance business plan persona user experience ...,Business,business strategy business model canvas analys...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[-1.62371062e-02 -5.21499477e-02 3.00578535e-...
2,0003,silicon thin film solar cell,�cole Polytechnique,Advanced,4.1,https://www.coursera.org/learn/silicon-thin-fi...,course consist general presentation solar cell...,chemistry physics solar energy film lambda cal...,Physical Science and Engineering,silicon thin film solar cell course consist ge...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[ 1.11733921e-01 2.74278939e-01 4.72732037e-...
3,0004,finance manager,IESE Business School,Intermediate,4.8,https://www.coursera.org/learn/operational-fin...,come number always meet eye operational financ...,account receivable dupont analysis analysis ac...,Business,finance manager come number always meet eye op...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[-4.35875319e-02 1.21336073e-01 3.25895369e-...
4,0005,retrieve datum use single table sql query,Coursera Project Network,Beginner,4.6,https://www.coursera.org/learn/single-table-sq...,course learn effectively retrieve datum relati...,datum analysis select sql database management ...,Information Technology,retrieve datum use single table sql query cour...,[117.0 34.0 27.0 33.0 258.0 201.0 42.0 0.0 125...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[ 6.04763143e-02 3.37209553e-02 4.53004211e-...


In [35]:
history = pd.read_csv("../data/history.csv", encoding='latin-1')

In [36]:
history # 395	user0396	

,User_id,History_course_id,History_course_name
0,user0001,"0687, 0355, 1677","Sequence Models, Natural Language Processing w..."
1,user0002,"0636, 0637, 1787","Exploratory Data Analysis, Exploratory Data An..."
2,user0003,"0120, 0286, 2712","Managing Big Data with MySQL, Introduction to ..."
3,user0004,"0406, 0603, 0840",Data Visualization and Communication with Tabl...
4,user0005,"0241, 2558, 3025, 3503",Deploy Models with TensorFlow Serving and Flas...
...,...,...,...
2050,user2051,"0843, 1189, 2399, 2573",NaN
2051,user2052,"1691, 2178, 2573, 2927",NaN
2052,user2053,"1296, 2118, 3059",NaN
2053,user2054,"1185, 1320, 2367, 3331",NaN


In [37]:
data = list(history['History_course_id'].apply(lambda x:x.split(",") ))

In [38]:
data

[['0687', ' 0355', ' 1677'],
 ['0636', ' 0637', ' 1787'],
 ['0120', ' 0286', ' 2712'],
 ['0406', ' 0603', ' 0840'],
 ['0241', ' 2558', ' 3025', ' 3503'],
 ['0450', ' 0662', ' 2696'],
 ['0391', ' 1808', ' 3454'],
 ['2533', ' 2543', ' 2564'],
 ['2919', ' 3097', ' 3260'],
 ['1581', ' 2154', ' 2725'],
 ['0088', ' 0112', ' 0233'],
 ['0211', ' 0220', ' 0241'],
 ['0149', ' 0211', ' 0288'],
 ['0220', ' 0241', ' 0288'],
 ['0199', ' 0211', ' 0241'],
 ['0120', ' 0286', ' 0413'],
 ['0066', ' 0149', ' 0396'],
 ['0108', ' 0279', ' 0687'],
 ['0330', ' 0376', ' 0530'],
 ['0058', ' 0221', ' 0849'],
 ['0485', ' 1210', ' 1792'],
 ['1366', ' 2486', ' 3352'],
 ['3399', ' 3423', ' 3454'],
 ['0543', ' 2854', ' 3352'],
 ['2845', ' 3349', ' 3361'],
 ['3352', ' 3369', ' 3399'],
 ['3334', ' 3352', ' 3361'],
 ['3361', ' 3369', ' 3370'],
 ['3361', ' 2708'],
 ['3352', ' 3369', ' 3376'],
 ['3352', ' 3369', ' 3423'],
 ['3454', ' 3465', ' 3472'],
 ['3465', ' 3472', ' 3475'],
 ['3475', ' 3483', ' 3491'],
 ['0636', ' 06

In [39]:
#Let's transform the list, with one-hot encoding
from mlxtend.preprocessing import TransactionEncoder
a = TransactionEncoder()
a_data = a.fit(data).transform(data)
df = pd.DataFrame(a_data,columns=a.columns_)
df = df.replace(False,0)
df

,0005,0005,0014,0022,0024,0031,0044,0045,0053,0062,...,3465,3471,3475,3494,3498,3500,3502,3504,3514,Data analysis
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2050,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2051,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2052,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2053,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
df2 = df

In [41]:
#set a threshold value for the support value and calculate the support value.
apriori_t = apriori(df2, min_support = 0.015, use_colnames = True, verbose = 1)
apriori_t

c:\Users\Sysme Hue\AppData\Local\Programs\Python\Python310\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


Processing 36 combinations | Sampling itemset size 43


,support,itemsets
0,0.016058,( 0816)
1,0.018491,( 0863)
2,0.018978,( 0992)
3,0.028224,( 1075)
4,0.016545,( 1109)
5,0.016058,( 1213)
6,0.029197,( 1289)
7,0.018978,( 1496)
8,0.035036,( 1502)
9,0.035036,( 1599)


In [42]:
#Let's view our interpretation values using the Associan rule function.
df_ar = association_rules(apriori_t, metric = "confidence", min_threshold = 0.6)
df_ar

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,( 1801),( 0992),0.015085,0.018978,0.015085,1.000000,52.692308,0.014799,inf,0.996047
1,( 0992),( 1801),0.018978,0.015085,0.015085,0.794872,52.692308,0.014799,4.801460,1.000000
2,( 2924),( 0992),0.019951,0.018978,0.015085,0.756098,39.840525,0.014707,4.022190,0.994746
3,( 0992),( 2924),0.018978,0.019951,0.015085,0.794872,39.840525,0.014707,4.777737,0.993760
4,( 3278),( 1075),0.018491,0.028224,0.018491,1.000000,35.431034,0.017970,inf,0.990084
...,...,...,...,...,...,...,...,...,...,...
61,"(1075, 1502)","( 1599, 1496)",0.025304,0.018978,0.018978,0.750000,39.519231,0.018498,3.924088,1.000000
62,"( 1599, 1496)","(1075, 1502)",0.018978,0.025304,0.018978,1.000000,39.519231,0.018498,inf,0.993552
63,"( 1502, 1496)","(1075, 1599)",0.018978,0.025304,0.018978,1.000000,39.519231,0.018498,inf,0.993552
64,(1075),"( 1599, 1502, 1496)",0.025304,0.018978,0.018978,0.750000,39.519231,0.018498,3.924088,1.000000


In [43]:
df_ar[:60]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,( 1801),( 0992),0.015085,0.018978,0.015085,1.000000,52.692308,0.014799,inf,0.996047
1,( 0992),( 1801),0.018978,0.015085,0.015085,0.794872,52.692308,0.014799,4.801460,1.000000
2,( 2924),( 0992),0.019951,0.018978,0.015085,0.756098,39.840525,0.014707,4.022190,0.994746
3,( 0992),( 2924),0.018978,0.019951,0.015085,0.794872,39.840525,0.014707,4.777737,0.993760
4,( 3278),( 1075),0.018491,0.028224,0.018491,1.000000,35.431034,0.017970,inf,0.990084
5,( 1075),( 3278),0.028224,0.018491,0.018491,0.655172,35.431034,0.017970,2.846375,1.000000
6,( 1075),(0014),0.028224,0.028224,0.028224,1.000000,35.431034,0.027427,inf,1.000000
7,(0014),( 1075),0.028224,0.028224,0.028224,1.000000,35.431034,0.027427,inf,1.000000
8,( 1289),(0567),0.029197,0.019465,0.019465,0.666667,34.250000,0.018896,2.941606,1.000000
9,(0567),( 1289),0.019465,0.029197,0.019465,1.000000,34.250000,0.018896,inf,0.990074


In [44]:
result = course.loc[course['Course_id'] == "0992"]['Course Name'].iloc[0]
print('antecedents:', result)



antecedents: neural network deep learning


In [50]:
# Tạo một từ điển (dictionary) để lưu trữ tên của các khoá học dựa trên ID
course_dict = dict(zip(course['Course_id'], course['Course Name']))

for n, antecedents in enumerate(df_ar['antecedents']):
    if isinstance(antecedents, frozenset):
        antecedents = list(antecedents)
    antecedent_ids = ','.join(map(str, antecedents)).strip()  # Chuyển thành danh sách và tách các ID

    antecedent_names = []
    for antecedent_id in antecedent_ids.split(','):
        antecedent_id = antecedent_id.strip()
        antecedent_name = course_dict.get(antecedent_id, 'Not Found')  # Lấy tên từ từ điển
        antecedent_names.append(antecedent_name)

    antecedent_names_str = ', '.join(antecedent_names)

    # print(n)
    print('antecedents:', antecedent_names_str)

    consequents = df_ar['consequents'][n]
    if isinstance(consequents, frozenset):
        consequents = list(consequents)
    consequent_ids = ','.join(map(str, consequents)).strip()  # Chuyển thành danh sách và tách các ID

    consequent_names = []
    for consequent_id in consequent_ids.split(','):
        consequent_id = consequent_id.strip()
        consequent_name = course_dict.get(consequent_id, 'Not Found')  # Lấy tên từ từ điển
        consequent_names.append(consequent_name)

    consequent_names_str = ', '.join(consequent_names)

    print('consequents:', consequent_names_str, '\n', '-' * 30)


antecedents: predictive modelling azure machine learn studio
consequents: neural network deep learning 
 ------------------------------
antecedents: neural network deep learning
consequents: predictive modelling azure machine learn studio 
 ------------------------------
antecedents: machine learn regression
consequents: neural network deep learning 
 ------------------------------
antecedents: neural network deep learning
consequents: machine learn regression 
 ------------------------------
antecedents: datum analytic accounting capstone
consequents: advanced business analytic capstone 
 ------------------------------
antecedents: advanced business analytic capstone
consequents: datum analytic accounting capstone 
 ------------------------------
antecedents: advanced business analytic capstone
consequents: business statistic analysis capstone 
 ------------------------------
antecedents: business statistic analysis capstone
consequents: advanced business analytic capstone 
 ---------